In [ ]:
import pandas as pd
import io
import copy

In [ ]:
import os
os.listdir('/content/drive/MyDrive/aldadata/summaries/')
path15='/content/drive/MyDrive/aldadata/summaries/hard-words-list.csv'
df_difficult_words= pd.read_csv(path15)

path16='/content/drive/MyDrive/aldadata/summaries/GPT-wiki-intro.csv'
df_complete_dataset=pd.read_csv(path16)

df_complete_dataset


,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,44173767,https://en.wikipedia.org/wiki/Randy%20Borum,Randy Borum,Randy Borum is a Professor and Coordinator of ...,Randy Borum is a Professor and Coordinator of ...,2,185,71,200 word wikipedia style introduction on 'Rand...,of the Master of Fine Arts Program in Creativ...,25,92
149996,33564134,https://en.wikipedia.org/wiki/Sa%27och%20language,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nea...","Sa'och (, also, ""Sauch"") is an endangered, nuc...",2,175,134,200 word wikipedia style introduction on 'Sa'o...,"nuclear-speaking, isolate language of the Ath...",33,184
149997,4219548,https://en.wikipedia.org/wiki/Philip%20Hanawalt,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,Philip C. Hanawalt (born 1931) is an American ...,2,166,191,200 word wikipedia style introduction on 'Phil...,American graphic artist and illustrator. He i...,30,272
149998,2625970,https://en.wikipedia.org/wiki/Vossius%20Gymnasium,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in Ams...,Vossius Gymnasium is a public gymnasium in the...,2,168,108,200 word wikipedia style introduction on 'Voss...,"the town of Vossius, Netherlands. It is named...",32,147


In [ ]:
df_complete_dataset['wiki_intro']

0         Sexhow railway station was a railway station b...
1         In Finnish folklore, all places and things, an...
2         In mathematics, specifically differential calc...
3         is a Japanese shōjo manga series written and i...
4         Robert Milner "Rob" Bradley, Jr. (born August ...
                                ...                        
149995    Randy Borum is a Professor and Coordinator of ...
149996    Sa'och (, also, "Sauch") is an endangered, nea...
149997    Philip C. Hanawalt (born 1931) is an American ...
149998    Vossius Gymnasium is a public gymnasium in Ams...
149999    Simone Stratigo (, Symeon Filippos Stratigos; ...
Name: wiki_intro, Length: 150000, dtype: object

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
human_sentences = df_complete_dataset['wiki_intro'][:10]
for word in human_sentences:
  list_words=word.split(" ")
  for i in list_words:
    if i in df_difficult_words['abase']:
      df_complete_dataset.replace(i," ", inplace=True)
df_complete_dataset['wiki_intro']

df_complete_dataset.to_csv(index=False)


In [ ]:
df_complete_dataset['human_hard_word_count'].sum()

0

In [ ]:
path1='/content/drive/MyDrive/aldadata/summaries/ai_text_summaries_0_24000.csv'
path2='/content/drive/MyDrive/aldadata/summaries/ai_text_summaries_24000_55000.csv'
path3='/content/drive/MyDrive/aldadata/summaries/ai_text_summaries_55000_80000.csv'
path4='/content/drive/MyDrive/aldadata/summaries/ai_text_summaries_80000_100000.csv'
path5='/content/drive/MyDrive/aldadata/summaries/ai_text_summaries_100000_125000.csv'
path6='/content/drive/MyDrive/aldadata/summaries/ai_text_summaries_125000_150000.csv'

path7='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_0_20000.csv'
path8='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_20000_40000.csv'
path9='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_40000_60000.csv'
path10='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_60000_80000.csv'
path11='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_80000_100000.csv'
path12='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_100000_120000.csv'
path13='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_120000_135000.csv'
path14='/content/drive/MyDrive/aldadata/summaries/human_text_summaries_135000_150000.csv'



df = pd.concat(map(pd.read_csv, [path1, path2,path3,path4,path5,path6]))
df2=pd.concat(map(pd.read_csv, [path7, path8,path9,path10,path11,path12,path13,path14]))
df2.head(10)
#df.shape

,Summary No.,H_Summaries
0,1,Sexhow railway station was built to serve the ...
1,2,"In Finnish folklore, all places and things, an..."
2,3,The inverse function theorem gives a sufficien...
3,4,The series began serialization in Margaret mag...
4,5,"Robert Milner ""Rob"" Bradley, Jr. is a Republic..."
5,6,Moluccans are the Austronesian-speaking and Pa...
6,7,HarperOne is a publishing imprint of HarperCol...
7,8,A full employment theorem is a theorem which s...
8,9,"""Pussy Fairy (OTW)"" is a song recorded by Amer..."
9,10,Qasr Ibrahim (Ibrahim Palace) is a historical ...


In [ ]:
df_cd = pd.merge(df, df2, how='inner')
df_cd.shape

df_dataset=copy.deepcopy(df_cd)
df_dataset


for i in df_dataset['H_Summaries']:
  words=i.split(' ')
  for j in words:
    if j in df3['abase']:
      print('yes')


In [ ]:
import spacy

In [ ]:
list_ai= df_cd['AI_Summaries']
list_hsumm=df_cd['H_Summaries']
list_hsumm

0         Sexhow railway station was built to serve the ...
1         In Finnish folklore, all places and things, an...
2         The inverse function theorem gives a sufficien...
3         The series began serialization in Margaret mag...
4         Robert Milner "Rob" Bradley, Jr. is a Republic...
                                ...                        
149995    Randy Borum is a Professor and Coordinator of ...
149996    Sa'och is an endangered, nearly extinct Pearic...
149997    Philip C. Hanawalt is an American biologist wh...
149998    Vossius Gymnasium is a public gymnasium in Ams...
149999    Simone Stratigo was an Italian Greek mathemati...
Name: H_Summaries, Length: 150000, dtype: object

In [ ]:

import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
# embeddings_hsumm1 = embed(list_hsumm[:40000])
embeddings_hsumm1 = embed(list_hsumm[110000:150001])
print(embeddings_hsumm1)
embeddings_hsumm1.shape

tf.Tensor(
[[ 0.01984784 -0.04909109 -0.03318809 ... -0.01530186 -0.01012475
   0.05272123]
 [ 0.02128438  0.0712356   0.07640021 ...  0.04007152  0.06553328
  -0.00981622]
 [-0.03048722  0.01586401  0.05794827 ... -0.06080077 -0.02917061
  -0.06318729]
 ...
 [-0.02716357  0.06389368 -0.01079725 ...  0.01770742 -0.0353317
  -0.01649206]
 [-0.05683472  0.0467064  -0.00946664 ...  0.02233877 -0.06796696
  -0.077488  ]
 [-0.0237885  -0.01928458 -0.00502092 ... -0.02171561  0.0134673
  -0.07681747]], shape=(40000, 512), dtype=float32)


TensorShape([40000, 512])

In [ ]:
# embeddings_hsumm2=embed(list_hsumm[40000:80000])
# print(embeddings_hsumm2)
# embeddings_hsumm2.shape

In [ ]:
# embeddings_ai1 = embed(list_ai[:40000])
embeddings_ai1 = embed(list_ai[110000:150000])
print(embeddings_ai1)
embeddings_ai1.shape

tf.Tensor(
[[ 0.00786317  0.04805319 -0.02441321 ... -0.04943983 -0.06227077
   0.03188179]
 [ 0.04677891  0.01822777  0.07704028 ...  0.04760573  0.07726154
  -0.00996508]
 [-0.02235756 -0.03710325  0.05100926 ... -0.03742388 -0.04581688
  -0.03885078]
 ...
 [ 0.0001982   0.04889482  0.0734999  ... -0.06059496  0.01240921
   0.02514476]
 [-0.01534636  0.07355625 -0.06791458 ... -0.03986057 -0.0369526
  -0.02258362]
 [-0.03293296 -0.04012745 -0.05378958 ... -0.05972523 -0.06963068
  -0.07219719]], shape=(40000, 512), dtype=float32)


TensorShape([40000, 512])

In [ ]:
# embeddings_ai2 = embed(list_ai[40000:80000])
# print(embeddings_ai2)
# embeddings_ai2.shape

In [ ]:
import tensorflow as tf



In [ ]:
def convert_tensor_to_list(embeddings):
  proto_tensor = tf.make_tensor_proto(embeddings)  # convert `tensor a` to a proto tensor
  embed_list=tf.make_ndarray(proto_tensor)
  return embed_list

In [ ]:
embeddings_hsumm1=convert_tensor_to_list(embeddings_hsumm1)
print(embeddings_hsumm1)
embeddings_hsumm1.shape

[[ 0.01984784 -0.04909109 -0.03318809 ... -0.01530186 -0.01012475
   0.05272123]
 [ 0.02128438  0.0712356   0.07640021 ...  0.04007152  0.06553328
  -0.00981622]
 [-0.03048722  0.01586401  0.05794827 ... -0.06080077 -0.02917061
  -0.06318729]
 ...
 [-0.02716357  0.06389368 -0.01079725 ...  0.01770742 -0.0353317
  -0.01649206]
 [-0.05683472  0.0467064  -0.00946664 ...  0.02233877 -0.06796696
  -0.077488  ]
 [-0.0237885  -0.01928458 -0.00502092 ... -0.02171561  0.0134673
  -0.07681747]]


(40000, 512)

In [ ]:
# embeddings_hsumm2=convert_tensor_to_list(embeddings_hsumm2)
# print(embeddings_hsumm2)

embeddings_ai1=convert_tensor_to_list(embeddings_ai1)
print(embeddings_ai1)
# embeddings_ai2=convert_tensor_to_list(embeddings_ai2)
# print(embeddings_ai2)

# print(embeddings_hsumm2.shape)
print(embeddings_ai1.shape)
# print(embeddings_ai2.shape)

[[ 0.00786317  0.04805319 -0.02441321 ... -0.04943983 -0.06227077
   0.03188179]
 [ 0.04677891  0.01822777  0.07704028 ...  0.04760573  0.07726154
  -0.00996508]
 [-0.02235756 -0.03710325  0.05100926 ... -0.03742388 -0.04581688
  -0.03885078]
 ...
 [ 0.0001982   0.04889482  0.0734999  ... -0.06059496  0.01240921
   0.02514476]
 [-0.01534636  0.07355625 -0.06791458 ... -0.03986057 -0.0369526
  -0.02258362]
 [-0.03293296 -0.04012745 -0.05378958 ... -0.05972523 -0.06963068
  -0.07219719]]
(40000, 512)


In [ ]:
import numpy as np

def cos_sim(a,b):

  # define the two vectors
  A = np.array(a)
  B = np.array(b)

  # calculate the dot product of A and B
  dot_product = np.dot(A, B)

  # calculate the magnitudes of A and B
  magnitude_A = np.sqrt(np.dot(A, A))
  magnitude_B = np.sqrt(np.dot(B, B))

  # calculate the cosine similarity
  cosine_similarity = dot_product / (magnitude_A * magnitude_B)

  #print("The cosine similarity of A and B is:", cosine_similarity)
  return cosine_similarity

In [ ]:
count=0
for i in range(0,40000):

  sim_val=cos_sim(embeddings_hsumm1[i],embeddings_ai1[i])

  if sim_val>=0.8:
    count+=1
    # print(str(df['AI_Summaries'][i+11000]))
    # print(str(df2['H_Summaries'][i+110000]))

# for i in range(0,40000):

#   sim_val=cos_sim(embeddings_hsumm2[i],embeddings_ai2[i])

#   if sim_val>=0.8:
#     count+=1
  
print(count)


860
